In [ ]:
# check system specs

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print('Connected to a GPU')

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9

if ram_gb < 20:
  print('Not using a high-RAM runtime: {:.1f} gigabytes of available RAM'.format(ram_gb))
else:
  print('Using a high-RAM runtime: {:.1f} gigabytes of available RAM'.format(ram_gb))

Connected to a GPU
Using a high-RAM runtime: 89.6 gigabytes of available RAM


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# note: place shortcut to shared project folder in google drive root directory
%cd /content/gdrive/MyDrive/ml2_project

/content/gdrive/.shortcut-targets-by-id/1WHLBzPq6pt_F7mh3d3goIQl4MwYlTfIh/ml2_project


In [ ]:
!pip install sentence_transformers

In [ ]:
# load models

from sentence_transformers import SentenceTransformer
import torch

# base_model = SentenceTransformer('all-MiniLM-L12-v2')
# base_model.save('./all-MiniLM-L12-v2')
base_model = SentenceTransformer('./models/all-MiniLM-L12-v2')
base_model = base_model.to(torch.device('cuda')) # use GPU

ft_final_model = SentenceTransformer('./models/finetune_mnr_final')
ft_final_model = ft_final_model.to(torch.device('cuda')) # use GPU


### Example Embedding without SentenceTransformers


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)

### Small Dataset
Load small dataset and append embeddings from base model and final fine-tuned model

In [ ]:
sds = pd.read_csv('data/small_dataset.csv')

In [ ]:
embeddings_base = base_model.encode(sds['lyrics'])

In [ ]:
embeddings_ft_final = ft_final_model.encode(sds['lyrics'])

In [ ]:
sds['embeddings_base'] = list(embeddings_base)
sds['embeddings_ft_final'] = list(embeddings_ft_final)

In [ ]:
sds.to_csv('data/small_dataset_w_embeddings.csv')

### Large Dataset
Load full dataset and append embeddings from base model and final fine-tuned model

In [ ]:
df = pd.read_csv('data/clean_dataset.csv')

In [ ]:
embeddings_base = base_model.encode(df['lyrics'])

In [ ]:
embeddings_ft_final = ft_final_model.encode(df['lyrics'])

In [ ]:
df['embeddings_base'] = list(embeddings_base)
df['embeddings_ft_final'] = list(embeddings_ft_final)

In [ ]:
df.to_csv('data/clean_dataset_w_embeddings.csv')